# Pandas Draft Assistant

A code-based draft assistant (no UI).

In [2]:
import statisticaldrafting as sd

dm = sd.DraftModel(set="FDN")

In [7]:
# Draft assistant
collection = ["Vivien Reid", "Llanowar Elves", "Refute", "Bigfin Bouncer"] # Collection as cardnames
# collection = [16, 229, 78, 78, 198] # Collection as card ids
# collection = [] # Pack 1 pick 1
pick_order = dm.get_pick_order(collection)
pick_order = pick_order[pick_order["rarity"].isin(["common", "uncommon"])]
pick_order.head(10)

,name,rarity,color_identity,p1p1_rating,synergy,rating
174,Mischievous Mystic,uncommon,U,113.4,3.8,117.2
84,Elvish Regrower,uncommon,G,83.6,31.0,114.6
98,Felling Blow,uncommon,G,79.9,34.0,113.9
180,Nessian Hornbeetle,uncommon,G,71.0,36.9,107.9
94,Faebloom Trick,uncommon,U,103.3,3.9,107.2
163,Llanowar Elves,common,G,72.8,32.8,105.6
233,Spectral Sailor,uncommon,U,96.0,6.6,102.6
250,"Tatyova, Benthic Druid",uncommon,Multicolor,71.9,28.9,100.8
88,Essence Scatter,uncommon,U,87.0,11.4,98.4
171,Micromancer,uncommon,U,97.1,-1.0,96.1


In [ ]:
# Deckbuild assistant (plan)

# Export collection from Arena into notebook
sample_sealed = """
2 Healer's Hawk (FDN) 142
1 Mountain (FDN) 279
2 Joust Through (FDN) 19
1 Ajani's Pridemate (FDN) 135
1 Banishing Light (FDN) 138
1 Dazzling Angel (FDN) 9
1 Inspiring Paladin (FDN) 18
1 Armasaur Guide (FDN) 3
1 An Offer You Can't Refuse (FDN) 160
1 Swamp (FDN) 277
1 Witness Protection (FDN) 168
1 Aetherize (FDN) 151
1 Lightshell Duo (FDN) 157
1 Extravagant Replication (FDN) 154
1 Grappling Kraken (FDN) 39
3 Island (FDN) 275
1 Pilfer (FDN) 181
4 Plains (FDN) 273
1 Frenzied Goblin (FDN) 199
1 Kellan, Planar Trailblazer (FDN) 91
1 Rite of the Dragoncaller (FDN) 92
1 Mild-Mannered Librarian (FDN) 228
3 Forest (FDN) 281
1 Genesis Wave (FDN) 221
1 Scoured Barrens (FDN) 266
1 Dismal Backwater (FDN) 261
1 Swiftwater Cliffs (FDN) 268
1 Jungle Hollow (FDN) 263
1 Wardens of the Cycle (FDN) 125
1 Evolving Wilds (FDN) 262
1 Adventuring Gear (FDN) 249

Sideboard
1 Goblin Boarders (FDN) 87
2 Vampire Soulcaller (FDN) 75
1 Gutless Plunderer (FDN) 60
2 Think Twice (FDN) 165
1 Wary Thespian (FDN) 235
1 Serra Angel (FDN) 147
2 Garruk's Uprising (FDN) 220
1 Stromkirk Bloodthief (FDN) 185
1 Spinner of Souls (FDN) 112
2 Axgard Cavalry (FDN) 189
1 Dwynen's Elite (FDN) 218
1 Involuntary Employment (FDN) 203
1 Helpful Hunter (FDN) 16
2 Erudite Wizard (FDN) 37
1 Burnished Hart (FDN) 250
1 Blanchwood Armor (FDN) 213
1 Phyrexian Arena (FDN) 180
1 Inspiring Paladin (FDN) 18
1 Courageous Goblin (FDN) 82
2 Hungry Ghoul (FDN) 62
1 Prideful Parent (FDN) 21
2 Broken Wings (FDN) 214
1 Campus Guide (FDN) 251
1 Tragic Banshee (FDN) 73
1 Tatyova, Benthic Druid (FDN) 247
1 Inspiration from Beyond (FDN) 43
1 Genesis Wave (FDN) 221
1 Treetop Snarespinner (FDN) 114
2 Fanatical Firebrand (FDN) 195
1 Gleaming Barrier (FDN) 252
1 Bite Down (FDN) 212
1 Soul-Shackled Zombie (FDN) 70
1 Uncharted Voyage (FDN) 53
1 Wardens of the Cycle (FDN) 125
1 Slumbering Cerberus (FDN) 94
1 Drakuseth, Maw of Flames (FDN) 193
1 Strix Lookout (FDN) 52
1 Spitfire Lagac (FDN) 208
1 Stab (FDN) 71
1 Apothecary Stomper (FDN) 99
1 Infernal Vessel (FDN) 63
1 Tolarian Terror (FDN) 167
1 Icewind Elemental (FDN) 42
1 Cackling Prowler (FDN) 101
1 Cephalid Inkmage (FDN) 32
1 Youthful Valkyrie (FDN) 149
"""


# Parse into collection list
# Deckbuilding algorithm
# Show exportable deck